**Задание:** Построить модель для прогнозирования сердечной недостаточности.

[Описание датасета](https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction)

[Датасет](https://disk.yandex.ru/d/gCVRjDJiR1tpSg)

[Датасет для тестирования](https://disk.yandex.ru/d/9aX3UpcRlYAJIw)

**Требования:**

- оберните весь конвейер преобразований в Pipeline

- подберите оптимальный вариант прогнозной модели с помощью GridSearchCV

- примените обученный на тренировочных данных конвейер к тестовым данным, никак не предобрабатывая их, а лишь загрузив из файла и отделив целевой признак от остальных

- получите на тестовой части качество не ниже 0.87 по метрике ROCAUC

Пояснение: если пропуски в новых данных и будут, то только в тех колонках, где они есть в тренировочной части.

In [47]:
import pandas as pd

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator

from xgboost import XGBClassifier

Загрузим данные и ознакомимся с ними

In [2]:
train = pd.read_csv('https://raw.githubusercontent.com/carolinesofina/DS_sets/main/2.8/heart_adapt_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/carolinesofina/DS_sets/main/2.8/heart_adapt_test.csv')

In [4]:
train.sample(5)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
412,38.0,F,ASY,105.0,NaN,1,Normal,166,N,2.8,Up,1
332,43.0,F,TA,100.0,223.0,0,Normal,142,N,0.0,Up,0
293,54.0,F,ATA,120.0,221.0,0,Normal,138,N,1.0,Up,0
463,58.0,F,ATA,136.0,319.0,1,LVH,152,N,0.0,Up,1
350,NaN,M,ASY,132.0,227.0,1,ST,138,N,0.2,Up,0


In [12]:
train['HeartDisease'].value_counts()

HeartDisease
1    381
0    208
Name: count, dtype: int64

Классы в целевом признаке не сбалансированы

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 589 entries, 0 to 588
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             533 non-null    float64
 1   Sex             589 non-null    object 
 2   ChestPainType   589 non-null    object 
 3   RestingBP       588 non-null    float64
 4   Cholesterol     462 non-null    float64
 5   FastingBS       589 non-null    int64  
 6   RestingECG      589 non-null    object 
 7   MaxHR           589 non-null    int64  
 8   ExerciseAngina  589 non-null    object 
 9   Oldpeak         589 non-null    float64
 10  ST_Slope        589 non-null    object 
 11  HeartDisease    589 non-null    int64  
dtypes: float64(4), int64(3), object(5)
memory usage: 55.3+ KB


In [6]:
train.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,533.000000,588.000000,462.000000,589.000000,589.000000,589.000000,589.000000
mean,54.195122,133.358844,245.632035,0.258065,134.893039,0.937521,0.646859
std,9.532661,18.851852,58.599184,0.437942,24.942596,1.071318,0.478352
min,28.000000,80.000000,85.000000,0.000000,63.000000,-2.600000,0.000000
25%,48.000000,120.000000,209.000000,0.000000,117.000000,0.000000,0.000000
50%,55.000000,130.000000,240.000000,0.000000,135.000000,0.800000,1.000000
75%,61.000000,144.000000,279.750000,1.000000,154.000000,1.600000,1.000000
max,77.000000,200.000000,603.000000,1.000000,195.000000,5.000000,1.000000


Числовые признаки Age, RestingBP и Cholesterol содержат пропуски в данных, которые можно заполнить медианным значением. К категориальным признакам можно применить метод прямого кодирования.

Перед началом обработки отделим целовой признак от признаков

In [21]:
target_train = train['HeartDisease']
target_test = test['HeartDisease']

features_train = train.drop(['HeartDisease'], axis=1)
features_test = test.drop(['HeartDisease'], axis=1)

Напишем свой трансформер для заполнения пропусков в числовых признаках и прямого кодирования категориальных признаков


In [41]:
class DataTransform(TransformerMixin, BaseEstimator):

    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        for col in X.select_dtypes(include=['float64']).columns:
            df.loc[df.[col] == 0](df[col].median(), inplace=True)
        
        X = pd.get_dummies(X, dtype=int, drop_first=True)
        
        return X

Оборачиваем свой трансформер, скейлер и модель логистической регресии в пайплайн

In [39]:
pipe = Pipeline([  
    ('data_transform', DataTransform()),
    ('scaler', StandardScaler()),
    ('classify', LogisticRegression(class_weight='balanced', random_state=0))
])

Обучим нашу модель и проверим ее качество

In [46]:
pipe.fit(X=features_train, y=target_train)
print(f'Качество модели на обучающей выборке: {round(roc_auc_score(target_train, pipe.predict_proba(features_train)[:, 1]), 4)}')
print(f'Качество модели на тестовой выборке: {round(roc_auc_score(target_test, pipe.predict_proba(features_test)[:, 1]), 4)}')

Качество модели на обучающей выборке: 0.9401
Качество модели на тестовой выборке: 0.9053


Качество модели на тестовой выборке хорошее, но попробуем перебрать еще несколько моделей и параметров

In [53]:
params = [
    {'classify': [LogisticRegression(class_weight='balanced', random_state=0)]}, 
    {'classify': [DecisionTreeClassifier(class_weight='balanced', random_state=0)],
     'classify__max_depth': [2, 5, 10, 20]},
    {'classify': [XGBClassifier()],
     'classify__max_depth': [2, 5, 10], 'classify__n_estimators': [2, 3, 4],
     'classify__learning_rate': [0.1, 0.5, 1]}
]

grid_search = GridSearchCV(pipe, param_grid=params, cv=5, scoring='roc_auc')
grid_search.fit(X=features_train, y=target_train)
print(f'Качество модели на тестовой выборке c лучшей моделью: {round(roc_auc_score(target_test, grid_search.predict_proba(features_test)[:, 1]), 4)}')

Качество модели на тестовой выборке c лучшей моделью: 0.9053


Похоже, логистическая регрессия изначально была лучшим выбором.

In [55]:
grid_search.best_estimator_

Pipeline(steps=[('data_transform', DataTransform()),
                ('scaler', StandardScaler()),
                ('classify',
                 LogisticRegression(class_weight='balanced', random_state=0))])

**Вывод:** Лучше всего показала мебя модель логистиической регрессии со сзначеним метрики roc-auc на тестовой выборке равным 0.9053.